In [65]:
from ultralytics import YOLO
import cv2

In [66]:
# vid = cv2.VideoCapture(0)
# vid.set(cv2.CAP_PROP_AUTO_EXPOSURE, 0)
# vid.set(cv2.CAP_PROP_EXPOSURE, 250)
# vid.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc(*'MJPG'))
# vid.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
# vid.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
# vid.set(cv2.CAP_PROP_FPS, 30)

In [78]:
smoke_path = 'SmokeReal.png'
smoke_shape = (200, 200)
model_path = 'models/ryad288.pt'
video_path = 'res0.mp4'

In [79]:
smoke = cv2.imread(smoke_path)
smoke = cv2.resize(smoke, smoke_shape)
model = YOLO('models/ryad288.pt')
vid = cv2.VideoCapture(video_path)

In [82]:
while True:
    ret, frame = vid.read()
    if not ret:
        continue
    pred = model.predict(frame, conf=0.3, imgsz=288, verbose=False)[0]
    boxes = pred.boxes.xywh[:, :2].tolist()[:2]
    img = pred.orig_img
    for (x, y) in boxes:
        mask = (1 - (smoke > 0)).astype('uint8')
        x = int(x)
        y = int(y)
        
        y_min = max(0, y - smoke_shape[1] // 2)
        y_max = min(img.shape[0], y + smoke_shape[1] // 2)
        
        x_min = max(0, x - smoke_shape[0] // 2)
        x_max = min(img.shape[1], x + smoke_shape[0] // 2)
        
        delta_x_min = x - x_min
        delta_x_max = x_max - x
        delta_y_min = y - y_min
        delta_y_max = y_max - y
        
        crop_y = smoke_shape[1] // 2 - delta_y_min, smoke_shape[1]//2 + delta_y_max
        crop_x = smoke_shape[0] // 2 - delta_x_min, smoke_shape[0]//2 + delta_x_max
        
        mask = mask[crop_y[0] : crop_y[1], crop_x[0] : crop_x[1], :]
        new_smoke = smoke[crop_y[0] : crop_y[1], crop_x[0] : crop_x[1], :]
        
        # img[y_min:y_max, x_min:x_max, :] *= mask
        patch = img[y_min:y_max, x_min:x_max, :]
        patch_divided = patch * (1 - mask > 0) // 3
        patch_undivided = patch * (mask > 0)
        res = patch_divided + patch_undivided
        img[y_min:y_max, x_min:x_max, :] = res + new_smoke
        # img = cv2.circle(img, (int(x), int(y)), 20, (255, 0, 0), 10) 
    cv2.imshow('drift', img)

    key = cv2.waitKey(1)
    if key == 27:#if ESC is pressed, exit loop
        cv2.destroyAllWindows()
        break

KeyboardInterrupt: 

In [73]:
smoke.shape

(400, 400, 3)

In [41]:
crop_y, crop_x

((-3, 800), (0, 800))

In [42]:
new_smoke.shape

(3, 800, 3)